In [ ]:
# pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import seaborn as sns
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/a1rafi/SST2/main/raw_data_sst2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67349 entries, 0 to 67348
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idx       67349 non-null  int64 
 1   sentence  67349 non-null  object
 2   label     67349 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [ ]:
df

,idx,sentence,label
0,0,hide new secretions from the parental units,0
1,1,"contains no wit , only labored gags",0
2,2,that loves its characters and communicates som...,1
3,3,remains utterly satisfied to remain the same t...,0
4,4,on the worst revenge-of-the-nerds clichés the ...,0
...,...,...,...
67344,67344,a delightful comedy,1
67345,67345,"anguish , anger and frustration",0
67346,67346,"at achieving the modest , crowd-pleasing goals...",1
67347,67347,a patient viewer,1


## WE ARE USING NLTK LIBRARY FOR TOKENIZING THE SENTENCE



In [ ]:
with open('all_tokens.txt', 'w') as file:
    for idx, row in df.iterrows():
        text = row['sentence']
        tokens = word_tokenize(text)
        for token in tokens:
            file.write(token + '\n')

print("All tokens have been saved to all_tokens.txt")

All tokens have been saved to all_tokens.txt


In [ ]:
l1=[]

with open('all_tokens.txt', 'r') as file:
    for line in file:
        token = line.strip()
        l1.append(token)
        #print(token)
print(l1)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
special = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for i in special:
  if i not in l1:
    l1.append(i)
print(l1)

## Filtering any unknown character which is non-English or not in the allowed_chars to ov


In [ ]:
import string

allowed_chars = [
    '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
]


allowed_set = set(allowed_chars)

#Sample list of words
words = l1
words.append('héllo') #hovlle


def filter_word(word):
    return ''.join(char if char in allowed_set else 'ov' for char in word)

filtered_words = [filter_word(word) for word in words]

print(filtered_words)

Output hidden; open in https://colab.research.google.com to view.

## Substitution dictionary / table

In [ ]:
import random
import string

def create_mixed_substitution_dict(seed=42):
    # Combine all required characters
    chars = string.punctuation + string.digits + string.ascii_letters
    chars = list(chars)

    # Generate pairs of characters
    pairs = [a + b for a in chars for b in chars]

    # Use only pairs in the key
    key_combined = pairs.copy()

    # Set seed and shuffle the key for consistent results
    random.seed(seed)
    random.shuffle(key_combined)

    # Ensure no character or pair maps to itself
    valid_mapping = False
    while not valid_mapping:
        substitution_dict = {original: substitute for original, substitute in zip(chars + pairs, key_combined)}
        valid_mapping = all(original != substitute for original, substitute in substitution_dict.items())
        if not valid_mapping:
            random.shuffle(key_combined)

    return substitution_dict


def sub_encrypt(plain_text, substitution_dict):
    cipher_text = ""
    i = 0
    while i < len(plain_text):
        # Attempt to find a pair first
        if i + 1 < len(plain_text):
            pair = plain_text[i:i + 2]
            if pair in substitution_dict:
                cipher_text += substitution_dict[pair]
                i += 2
                continue

        # If no valid pair, fallback to single character
        char = plain_text[i]
        cipher_text += substitution_dict.get(char, char)
        i += 1

    return cipher_text

def clean_sentence(sentence):
    # Remove punctuation
    cleaned_sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    cleaned_sentence = cleaned_sentence.lower()
    return cleaned_sentence

substitution_dict = create_mixed_substitution_dict()


## Encrypting tokens

In [ ]:
encryptedTokens = []
Ctext = []
for plain_text in filtered_words:
    cipher_text = ""
    #for letter in plain_text:
        #cipher_text += substitution_dict.get(letter, "ov") #If letter is not a key in the dictionary, it returns "ov" as the default value. (just in case)
    cipher_text += sub_encrypt(clean_sentence(plain_text),substitution_dict)
    encryptedTokens.append((clean_sentence(plain_text), cipher_text))
    Ctext.append(cipher_text)

#print(encryptedTokens)
encryptedTokens100 = encryptedTokens[:100]
for plain_text, cipher_text in encryptedTokens100:
    print(f"Plain text: {plain_text} \nCipher text: {cipher_text}")
    print("==================")

Plain text: hide 
Cipher text: e#^;
Plain text: new 
Cipher text: [:tK
Plain text: secretions 
Cipher text: ?n#bxizhI%
Plain text: from 
Cipher text: ^HQ[
Plain text: the 
Cipher text: Q+EZ
Plain text: parental 
Cipher text: YT.Tr^&I
Plain text: units 
Cipher text: EOzG0L
Plain text: contains 
Cipher text: @-r^|)I%
Plain text: no 
Cipher text: c9
Plain text: wit 
Cipher text: 0E]O
Plain text:  
Cipher text: 
Plain text: only 
Cipher text: 17ug
Plain text: labored 
Cipher text: mo,8.Tn_
Plain text: gags 
Cipher text: .h}Y
Plain text: that 
Cipher text: Q+T0
Plain text: loves 
Cipher text: qtJ_0L
Plain text: its 
Cipher text: zG0L
Plain text: characters 
Cipher text: %aBQ.Dz-&9
Plain text: and 
Cipher text: IEn_
Plain text: communicates 
Cipher text: @-N=EODjT0zj
Plain text: something 
Cipher text: =*fGQ+3evT
Plain text: rather 
Cipher text: TlQ+ZW
Plain text: beautiful 
Cipher text: C33WV)k7>u
Plain text: about 
Cipher text: i{=`]O
Plain text: human 
Cipher text: ?!UkA"
Plain text: natu

## Remove duplicates

In [ ]:
#print(Ctext)
mylist = list(set(Ctext))
#print(mylist)
print("Total cypher tokens: ", len(mylist))

Total cypher tokens:  14706


In [ ]:
with open('all_bi_cypher_tokens.txt', 'w') as file:
  for cipher_text in mylist:
    file.write(cipher_text + '\n')